In [1]:
import numpy as np
import torch
from seqm.seqm_functions.constants import Constants
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin
from seqm.Molecule import Molecule

from seqm.seqm_functions.read_xyz import read_xyz

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [2]:
%%time
torch.manual_seed(0)
files = ['coronene.xyz']

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]
const = Constants().to(device)
#may need to add scaling factor for length and energy on const, check constants.py

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }
#
output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'Test1_aimd'}
mol = Molecule(const, seqm_parameters, coordinates, species).to(device)

md2 =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md2.initialize_velocity(mol )
#remove center of mass velocity
md=md2

_ = md.run(mol, 3, remove_com=[True, 1], Info_log=True)

/home/maxim/anaconda3/envs/pyseqm_4/lib/python3.9/site-packages/PYSEQM-0.0.1-py3.9.egg/seqm/seqm_functions/diag.py:134: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at /opt/conda/conda-bld/pytorch_1666642975993/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2794.)
  e0,v = torch.symeig(x0,eigenvectors=True,upper=True)


Step,    Temp,    E(kinetic),  E(potential),  E(total),      dipole(x,y,z)
     1   413.18   1.922664e+00 4.273019e+00 6.195684e+00   2.196748e-04 3.451560e-03 -4.993308e-03
     2   396.08   1.843116e+00 4.353703e+00 6.196819e+00   4.693076e-04 6.552232e-03 -9.721843e-03
     3   371.92   1.730698e+00 4.467521e+00 6.198220e+00   7.426086e-04 9.048870e-03 -1.389911e-02
CPU times: user 25.7 s, sys: 1.57 s, total: 27.3 s
Wall time: 4.69 s
